# Load segmentation, localise and save as h5

Segment a stack of images and then manually label a couple, then check how well the model segmented them.

In [1]:
import napari
import cellpose
from octopuslite import utils, tile
import numpy as np

import sys
sys.path.append('macrohet/')
from notify import send_sms

def view(img):
    return napari.Viewer().add_image(img)

from tqdm.auto import tqdm

import btrack
import dask.array as da

### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [2]:
image_dir = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Images/'
metadata_fn = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Index.idx.xml'
metadata = utils.read_harmony_metadata(metadata_fn)

Reading metadata XML file...


Extracting HarmonyV5 metadata:   0%|          | 0/113400 [00:00<?, ?it/s]

Extracting metadata complete!


### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [3]:
metadata_path = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml'
utils.read_harmony_metadata(metadata_path, assay_layout=True)

Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC
3 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
4 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
5 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99
6 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99

### Define row and column of choice

In [4]:
row = '6'
column = '9'

### Now to lazily mosaic the images using Dask prior to viewing them.

1x (75,2,3) [TCZ] image stack takes approximately 1 minute to stitch together, so only load the one field of view I want.

In [23]:
images = tile.compile_mosaic(image_dir, 
                             metadata, 
                             row, 
                             column, 
                             #set_channel=1, 
                             set_plane = 'sum_proj',
#                              set_time = 1
                         )#.astype(uint8)

#### Reorder the channel axis for localisation

In [24]:
gfp = images[:,0,...]
rfp = images[:,1,...]
images = da.stack([gfp,rfp], axis = -1)
images

,Array,Chunk
Bytes,40.88 GiB,31.01 MiB
Shape,"(75, 6048, 6048, 2)","(1, 2016, 2016, 1)"
Count,18900 Tasks,1350 Chunks
Type,uint64,numpy.ndarray


### Load masks

In [8]:
masks = np.load('segmentation/masks.npy',)

In [9]:
### convert to dask array so that it is compatible with images for localisation
masks = da.from_array(masks)

In [10]:
masks

dask.array<array, shape=(75, 6048, 6048), dtype=uint16, chunksize=(75, 864, 864), chunktype=numpy.ndarray>

# Localise raw masks

Include size filter and measurements of fluorescence

In [11]:
feat = [
      "area",
      "major_axis_length",
      "minor_axis_length",
      "orientation",
      "mean_intensity",
        ]

In [25]:
objects = btrack.utils.segmentation_to_objects(
    masks, 
    images,
    properties = tuple(feat),
    use_weighted_centroid = False, 
)

[INFO][2023/01/19 04:56:11 PM] Localizing objects from segmentation...
[INFO][2023/01/19 04:56:11 PM] Found intensity_image data
[INFO][2023/01/19 05:12:21 PM] Objects are of type: <class 'dict'>
[INFO][2023/01/19 05:12:21 PM] ...Found 89235 objects in 75 frames.


In [26]:
objects = [o for o in objects if o.properties['area'] > 2500]

#### Find the efd?

In [ ]:
### finding the EFD
for obj in tqdm(objects):
    ### extract the intensity image (1ch only)
    glimpse = obj.properties['intensity_image'][...,0]
    ### pad the glimpse to ensure only one object is identifiable
    glimpse = np.pad(glimpse, pad_width = 1)
    ### find the contours (zero because only one object)
    contours = skimage.measure.find_contours(glimpse, fully_connected='high', level = 0.5)[0]
    ### get the efd
    efd = elliptic_fourier_descriptors(contours, order=100, normalize=True)
#     obj.properties = {'efd': efd}
    flatten_efd = efd.flatten()
    obj.properties = {'efd flat': flatten_efd}

In [27]:
with btrack.dataio.HDF5FileHandler(
     'objects.h5', 'w', obj_type='obj_type_1',
) as hdf:
    hdf.write_segmentation(masks)
    hdf.write_objects(objects)

[INFO][2023/01/19 05:12:22 PM] Opening HDF file: objects.h5...
[INFO][2023/01/19 05:13:04 PM] Writing objects/obj_type_1
[INFO][2023/01/19 05:13:04 PM] Writing labels/obj_type_1
[INFO][2023/01/19 05:13:04 PM] Loading objects/obj_type_1 (40381, 5) (40381 filtered: None)
[INFO][2023/01/19 05:13:09 PM] Writing properties/obj_type_1/area (40381,)
[INFO][2023/01/19 05:13:09 PM] Writing properties/obj_type_1/major_axis_length (40381,)
[INFO][2023/01/19 05:13:09 PM] Writing properties/obj_type_1/minor_axis_length (40381,)
[INFO][2023/01/19 05:13:09 PM] Writing properties/obj_type_1/orientation (40381,)
[INFO][2023/01/19 05:13:09 PM] Writing properties/obj_type_1/mean_intensity-0 (40381,)
[INFO][2023/01/19 05:13:09 PM] Writing properties/obj_type_1/mean_intensity-1 (40381,)
[INFO][2023/01/19 05:13:09 PM] Closing HDF file: objects.h5
